In [1]:
from pandas import read_csv
from matplotlib import pyplot
import sys
import numpy as np
import pandas as pd
from datetime import datetime


#DATOS PISOA
#flat="./pisoA/"
#day="19244"
#folders=["TRACK2","TRACK3","TRACK4","TRACK5","TRACK6","TRACK7","TRACK8","TRACK9"]

#DATOS PISOB
flat="./pisoB/"
day="19311"
folders=["TRACK1","TRACK2","TRACK3","TRACK4","TRACK5","TRACK6","TRACK7","TRACK8","TRACK9"]


#Ventanas temporales de valores "pasados" (los datos RSSI despues de la estimación de tiempo) hasta 12 segundos
tW1=[0,-1,-2,-3,-4,-6,-8,-10,-12]





#LEEMOS los datos RSSI (tiempo, ancla y señal)
data=[]
N=0
for f in folders:
    series=read_csv(flat+f+"/uwb/"+day+".rssi.1.tsv", sep="\t",parse_dates=True,header=None)
    N=N+len(series.values)
    d2=series.values
    d2=np.array(d2)
    print(d2)
    print(d2[:,2])
    data.extend(d2)

data=np.array(data)
print(len(data))
print(N)


[[1668519247 '2022-11-15 14:34:07' 4908 -99.35]
 [1668519247 '2022-11-15 14:34:07' 8139 -94.58]
 [1668519247 '2022-11-15 14:34:07' 28079 -83.06]
 ...
 [1668519447 '2022-11-15 14:37:27' 28079 -83.82]
 [1668519447 '2022-11-15 14:37:27' 31176 -85.11]
 [1668519447 '2022-11-15 14:37:27' 38719 -97.27]]
[4908 8139 28079 31176 8139 28079 31176 4842 4908 8139 28079 31176 38719
 4908 8139 28079 31176 4842 4908 8139 28079 31176 38719 4842 4908 8139
 28079 31176 38719 4842 4908 8139 28079 31176 38719 4842 4908 8139 28079
 31176 38719 4842 4908 8139 28079 31176 38719 4842 4908 8139 28079 31176
 38719 4842 4908 8139 28079 31176 38719 4842 4908 8139 28079 31176 38719
 4842 4908 8139 28079 31176 38719 4842 4908 8139 28079 31176 38719 4842
 4908 8139 28079 31176 38719 4842 4908 8139 28079 31176 38719 4842 4908
 8139 28079 31176 38719 4842 4908 8139 28079 31176 38719 4842 4908 8139
 28079 31176 38719 4842 4908 8139 28079 31176 38719 4842 4908 8139 28079
 31176 38719 4842 4908 8139 28079 31176 38719 4842

In [2]:
#Leemos los datos del ground truth que ha etiquetado el usuario (pixeles en la imagen)
dataY=[]

for f in folders:
    series=read_csv(flat+f+"/location_gt/"+day+".location.tsv", sep="\t",parse_dates=True,header=None)
    for d in series.values:
        dataY.append(d)
dataY=np.array(dataY)

In [3]:
#Leemos las diferentes anclas (4) que hay configuradas
print(data[:,2])
anchors=np.unique(data[:,2])
print(anchors)

[4908 8139 28079 ... 28079 31176 38719]
[4842 4908 8139 28079 31176 38719]


In [4]:
#Normalizamos los valores de RSSI
min_v=(float)(min(data[:,3]))
max_v=(float)(max(data[:,3]))
print(min_v,max_v)

-120.0 -78.9


In [5]:
#INFO: tiempo inicial y final de los datos
min_t=(float)(min(dataY[:,0]))
max_t=(float)(max(dataY[:,0]))
print(min_t,max_t)

1668519288.0 1668521680.0


In [6]:
#INFO: valores min max de la Y
min_y=(float)(min(dataY[:,4]))
max_y=(float)(max(dataY[:,4]))
print(min_y,max_y)

66.0 675.0


In [7]:
#INFO: valores min max de la X
min_x=(float)(min(dataY[:,3]))
max_x=(float)(max(dataY[:,3]))
print(min_x,max_x)

95.0 493.0


In [8]:
#Esta función obtiene la media de un cojunto de señales RSSI (normalizada)
def getAVG(vs):
    if (len(vs)==0):
        return 0
    #print(" avg:", vs)
    return (np.mean(vs)-min_v)/(max_v-min_v)

#Esta función obtiene el MAX de un cojunto de señales RSSI (normalizada)
def getMAX(vs):
    if (len(vs)==0):
        return 0
    #print(" avg:", vs)
    return (max(vs)-min_v)/(max_v-min_v)

#Esta función obtiene el MIN de un cojunto de señales RSSI (normalizada)
def getMIN(vs):
    if (len(vs)==0):
        return 0
    #print(" avg:", vs)
    return (min(vs)-min_v)/(max_v-min_v)

In [9]:
#Valores de X, Y
X=[]
Y=[]



c=0

#Desde el tiempo inicial al final (en segundos) de uno en uno (con un desplazamiento en funcion del tamaño de ventana para recoger datos anteriores y futuros)
for t in np.arange(min_t-min(tW1),max_t,1.0):
    print(t)
    
    #Recogemos la posicioón de X,Y más cercana anterior y posterior a este tiempo t
    agXY0=dataY[np.sort(np.where( (dataY[:,0] <  t)))][0][-1]
    agXYNn=dataY[np.sort(np.where( (dataY[:,0] >=  t)))][0][0]
    #calculamos el incremento de tiempo respecto t, del futuro y del pasado
    incT0=t-agXY0[0]
    incTN=agXYNn[0]-t
    print("agXY0:",agXY0, "incT0:",incT0)
    print("agXYNn:",agXYNn, "incTN:",incTN)
    
    #IMPORTANTE: si el usuario no etiquetó esa muestra en un tiempo cercano (a 10 segundos), la descartamos!
    if(incT0>5 or incTN> 5):
        print("NOT DATA AVAILABLE!")
        continue
    
    #valores de las x-> NO posicion, los datos de entrada de RSSI
    xi=[]
    for a in anchors:
        xirssi1=[]
        xirssi2=[]
        xirssi3=[]
        xirssi4=[]
        xirssi5=[]
        xirssi6=[]
        #Ventanas pasadas
        for it in range(len(tW1)-2,-1,-1):
            t0=t+tW1[it]
            tN=t+tW1[it+1]
             #Todos los datos en ese segmento de ventana
            ag1=data[np.where((data[:,2] == a) & (data[:,0] <= t0) & (data[:,0] >=  tN))]
            #print("t+:",t,"a:",str(a),"t0:",t0,"tN:",tN," ag1:", ag1)
            #Sacamos las 3 características de esa ventana (max,min,media)
            xirssi1.append(getAVG(ag1[:,3]))
            xirssi2.append(getMAX(ag1[:,3]))
            xirssi3.append(getMIN(ag1[:,3]))

            
        #añadimos los valores de entradas futuras y pasadas a la sequencia     
        xi.append(xirssi1)
        xi.append(xirssi2)
        xi.append(xirssi3)

    

    
    #calculamos una posición X,Y de la imagen ponderada respecto las posiciones más cercana anterior y posterior a este tiempo t
    w0= 1-(incT0)/(incT0+incTN)
    wn= 1-(incTN)/(incT0+incTN)
    
    posX=(agXYNn[3]*wn+agXY0[3]*w0)
    posY=(agXYNn[4]*wn+agXY0[4]*w0)
    print("posX:",posX, "posY:",posY)
    
    #normalizamos 
    posX=(posX-min_x)/(max_x-min_x)
    posY=(posY-min_y)/(max_y-min_y)
    print("posX:",posX, "posY:",posY)
    
    #añadimos la sequencia de caracteristicas X (RSSI) e Y(posX,posY de la imagen)
    X.append(np.transpose(xi))
    Y.append([posX,posY])
    

    
    
    print("count:",c)



    c=c+1
    if(c>5000):
        break
        
        
#Importante: pasar los datos a NUMPY para trabajar con el modelo de DL
X=np.array(X)
Y=np.array(Y)
print(X.shape)
print(Y.shape)


1668519300.0
agXY0: [1668519289 '2022-11-15 14:34:49' 1668519289 415 97] incT0: 11.0
agXYNn: [1668519306 '2022-11-15 14:35:06' 1668519306 407 116] incTN: 6.0
NOT DATA AVAILABLE!
1668519301.0
agXY0: [1668519289 '2022-11-15 14:34:49' 1668519289 415 97] incT0: 12.0
agXYNn: [1668519306 '2022-11-15 14:35:06' 1668519306 407 116] incTN: 5.0
NOT DATA AVAILABLE!
1668519302.0
agXY0: [1668519289 '2022-11-15 14:34:49' 1668519289 415 97] incT0: 13.0
agXYNn: [1668519306 '2022-11-15 14:35:06' 1668519306 407 116] incTN: 4.0
NOT DATA AVAILABLE!
1668519303.0
agXY0: [1668519289 '2022-11-15 14:34:49' 1668519289 415 97] incT0: 14.0
agXYNn: [1668519306 '2022-11-15 14:35:06' 1668519306 407 116] incTN: 3.0
NOT DATA AVAILABLE!
1668519304.0
agXY0: [1668519289 '2022-11-15 14:34:49' 1668519289 415 97] incT0: 15.0
agXYNn: [1668519306 '2022-11-15 14:35:06' 1668519306 407 116] incTN: 2.0
NOT DATA AVAILABLE!
1668519305.0
agXY0: [1668519289 '2022-11-15 14:34:49' 1668519289 415 97] incT0: 16.0
agXYNn: [1668519306 '2022

posX: 183.0 posY: 111.0
posX: 0.22110552763819097 posY: 0.07389162561576355
count: 63
1668519374.0
agXY0: [1668519373 '2022-11-15 14:36:13' 1668519373 201 97] incT0: 1.0
agXYNn: [1668519374 '2022-11-15 14:36:14' 1668519374 201 96] incTN: 0.0
posX: 201.0 posY: 96.0
posX: 0.2663316582914573 posY: 0.04926108374384237
count: 64
1668519375.0
agXY0: [1668519374 '2022-11-15 14:36:14' 1668519374 201 96] incT0: 1.0
agXYNn: [1668519375 '2022-11-15 14:36:15' 1668519375 203 95] incTN: 0.0
posX: 203.0 posY: 95.0
posX: 0.271356783919598 posY: 0.047619047619047616
count: 65
1668519376.0
agXY0: [1668519375 '2022-11-15 14:36:15' 1668519375 215 93] incT0: 1.0
agXYNn: [1668519376 '2022-11-15 14:36:16' 1668519376 218 91] incTN: 0.0
posX: 218.0 posY: 91.0
posX: 0.30904522613065327 posY: 0.041050903119868636
count: 66
1668519377.0
agXY0: [1668519376 '2022-11-15 14:36:16' 1668519376 219 90] incT0: 1.0
agXYNn: [1668519377 '2022-11-15 14:36:17' 1668519377 219 90] incTN: 0.0
posX: 219.0 posY: 90.0
posX: 0.31155

posX: 104.0 posY: 629.0
posX: 0.022613065326633167 posY: 0.9244663382594417
count: 100
1668519411.0
agXY0: [1668519410 '2022-11-15 14:36:50' 1668519410 104 629] incT0: 1.0
agXYNn: [1668519411 '2022-11-15 14:36:51' 1668519411 118 629] incTN: 0.0
posX: 118.0 posY: 629.0
posX: 0.05778894472361809 posY: 0.9244663382594417
count: 101
1668519412.0
agXY0: [1668519411 '2022-11-15 14:36:51' 1668519411 130 626] incT0: 1.0
agXYNn: [1668519412 '2022-11-15 14:36:52' 1668519412 137 625] incTN: 0.0
posX: 137.0 posY: 625.0
posX: 0.10552763819095477 posY: 0.9178981937602627
count: 102
1668519413.0
agXY0: [1668519412 '2022-11-15 14:36:52' 1668519412 151 623] incT0: 1.0
agXYNn: [1668519413 '2022-11-15 14:36:53' 1668519413 152 623] incTN: 0.0
posX: 152.0 posY: 623.0
posX: 0.14321608040201006 posY: 0.9146141215106732
count: 103
1668519414.0
agXY0: [1668519413 '2022-11-15 14:36:53' 1668519413 151 622] incT0: 1.0
agXYNn: [1668519414 '2022-11-15 14:36:54' 1668519414 150 621] incTN: 0.0
posX: 150.0 posY: 621.0

posX: 287.0 posY: 537.0
posX: 0.4824120603015075 posY: 0.7733990147783252
count: 124
1668519562.0
agXY0: [1668519561 '2022-11-15 14:39:21' 1668519561 287 537] incT0: 1.0
agXYNn: [1668519565 '2022-11-15 14:39:25' 1668519565 290 530] incTN: 3.0
posX: 287.75 posY: 535.25
posX: 0.4842964824120603 posY: 0.7705254515599343
count: 125
1668519563.0
agXY0: [1668519561 '2022-11-15 14:39:21' 1668519561 287 537] incT0: 2.0
agXYNn: [1668519565 '2022-11-15 14:39:25' 1668519565 290 530] incTN: 2.0
posX: 288.5 posY: 533.5
posX: 0.4861809045226131 posY: 0.7676518883415435
count: 126
1668519564.0
agXY0: [1668519561 '2022-11-15 14:39:21' 1668519561 287 537] incT0: 3.0
agXYNn: [1668519565 '2022-11-15 14:39:25' 1668519565 290 530] incTN: 1.0
posX: 289.25 posY: 531.75
posX: 0.4880653266331658 posY: 0.7647783251231527
count: 127
1668519565.0
agXY0: [1668519561 '2022-11-15 14:39:21' 1668519561 287 537] incT0: 4.0
agXYNn: [1668519565 '2022-11-15 14:39:25' 1668519565 290 530] incTN: 0.0
posX: 290.0 posY: 530.0


posX: 100.66666666666669 posY: 576.0
posX: 0.014237855946398708 posY: 0.8374384236453202
count: 158
1668519599.0
agXY0: [1668519596 '2022-11-15 14:39:56' 1668519596 102 586] incT0: 3.0
agXYNn: [1668519599 '2022-11-15 14:39:59' 1668519599 100 571] incTN: 0.0
posX: 100.0 posY: 571.0
posX: 0.01256281407035176 posY: 0.8292282430213465
count: 159
1668519600.0
agXY0: [1668519599 '2022-11-15 14:39:59' 1668519599 100 571] incT0: 1.0
agXYNn: [1668519607 '2022-11-15 14:40:07' 1668519607 107 552] incTN: 7.0
NOT DATA AVAILABLE!
1668519601.0
agXY0: [1668519599 '2022-11-15 14:39:59' 1668519599 100 571] incT0: 2.0
agXYNn: [1668519607 '2022-11-15 14:40:07' 1668519607 107 552] incTN: 6.0
NOT DATA AVAILABLE!
1668519602.0
agXY0: [1668519599 '2022-11-15 14:39:59' 1668519599 100 571] incT0: 3.0
agXYNn: [1668519607 '2022-11-15 14:40:07' 1668519607 107 552] incTN: 5.0
posX: 102.625 posY: 563.875
posX: 0.019158291457286432 posY: 0.8175287356321839
count: 160
1668519603.0
agXY0: [1668519599 '2022-11-15 14:39:5

...

posX: 341.33333333333337 posY: 448.66666666666674
posX: 0.6189279731993301 posY: 0.628352490421456
count: 910
1668521520.0
agXY0: [1668521517 '2022-11-15 15:11:57' 1668521517 332 454] incT0: 3.0
agXYNn: [1668521520 '2022-11-15 15:12:00' 1668521520 346 446] incTN: 0.0
posX: 346.0 posY: 446.0
posX: 0.6306532663316583 posY: 0.6239737274220033
count: 911
1668521521.0
agXY0: [1668521520 '2022-11-15 15:12:00' 1668521520 346 459] incT0: 1.0
agXYNn: [1668521521 '2022-11-15 15:12:01' 1668521521 347 466] incTN: 0.0
posX: 347.0 posY: 466.0
posX: 0.6331658291457286 posY: 0.6568144499178982
count: 912
1668521522.0
agXY0: [1668521521 '2022-11-15 15:12:01' 1668521521 335 503] incT0: 1.0
agXYNn: [1668521522 '2022-11-15 15:12:02' 1668521522 338 532] incTN: 0.0
posX: 338.0 posY: 532.0
posX: 0.6105527638190955 posY: 0.7651888341543513
count: 913
1668521523.0
agXY0: [1668521522 '2022-11-15 15:12:02' 1668521522 344 570] incT0: 1.0
agXYNn: [1668521523 '2022-11-15 15:12:03' 1668521523 346 574] incTN: 0.0
pos

posX: 367.0 posY: 125.0
posX: 0.6834170854271356 posY: 0.09688013136288999
count: 937
1668521558.0
agXY0: [1668521557 '2022-11-15 15:12:37' 1668521557 346 156] incT0: 1.0
agXYNn: [1668521558 '2022-11-15 15:12:38' 1668521558 347 159] incTN: 0.0
posX: 347.0 posY: 159.0
posX: 0.6331658291457286 posY: 0.15270935960591134
count: 938
1668521559.0
agXY0: [1668521558 '2022-11-15 15:12:38' 1668521558 351 164] incT0: 1.0
agXYNn: [1668521559 '2022-11-15 15:12:39' 1668521559 342 177] incTN: 0.0
posX: 342.0 posY: 177.0
posX: 0.6206030150753769 posY: 0.18226600985221675
count: 939
1668521560.0
agXY0: [1668521559 '2022-11-15 15:12:39' 1668521559 336 191] incT0: 1.0
agXYNn: [1668521560 '2022-11-15 15:12:40' 1668521560 336 196] incTN: 0.0
posX: 336.0 posY: 196.0
posX: 0.6055276381909548 posY: 0.2134646962233169
count: 940
1668521561.0
agXY0: [1668521560 '2022-11-15 15:12:40' 1668521560 337 222] incT0: 1.0
agXYNn: [1668521561 '2022-11-15 15:12:41' 1668521561 337 226] incTN: 0.0
posX: 337.0 posY: 226.0
p

posX: 104.0 posY: 544.0
posX: 0.022613065326633167 posY: 0.7848932676518884
count: 971
1668521592.0
agXY0: [1668521591 '2022-11-15 15:13:11' 1668521591 104 544] incT0: 1.0
agXYNn: [1668521592 '2022-11-15 15:13:12' 1668521592 104 544] incTN: 0.0
posX: 104.0 posY: 544.0
posX: 0.022613065326633167 posY: 0.7848932676518884
count: 972
1668521593.0
agXY0: [1668521592 '2022-11-15 15:13:12' 1668521592 103 551] incT0: 1.0
agXYNn: [1668521593 '2022-11-15 15:13:13' 1668521593 116 547] incTN: 0.0
posX: 116.0 posY: 547.0
posX: 0.052763819095477386 posY: 0.7898193760262726
count: 973
1668521594.0
agXY0: [1668521593 '2022-11-15 15:13:13' 1668521593 116 547] incT0: 1.0
agXYNn: [1668521594 '2022-11-15 15:13:14' 1668521594 116 547] incTN: 0.0
posX: 116.0 posY: 547.0
posX: 0.052763819095477386 posY: 0.7898193760262726
count: 974
1668521595.0
agXY0: [1668521594 '2022-11-15 15:13:14' 1668521594 126 554] incT0: 1.0
agXYNn: [1668521595 '2022-11-15 15:13:15' 1668521595 134 568] incTN: 0.0
posX: 134.0 posY: 56

posX: 378.0 posY: 330.0
posX: 0.7110552763819096 posY: 0.43349753694581283
count: 1008
1668521629.0
agXY0: [1668521628 '2022-11-15 15:13:48' 1668521628 375 335] incT0: 1.0
agXYNn: [1668521629 '2022-11-15 15:13:49' 1668521629 374 337] incTN: 0.0
posX: 374.0 posY: 337.0
posX: 0.7010050251256281 posY: 0.44499178981937604
count: 1009
1668521630.0
agXY0: [1668521629 '2022-11-15 15:13:49' 1668521629 340 379] incT0: 1.0
agXYNn: [1668521630 '2022-11-15 15:13:50' 1668521630 338 383] incTN: 0.0
posX: 338.0 posY: 383.0
posX: 0.6105527638190955 posY: 0.5205254515599343
count: 1010
1668521631.0
agXY0: [1668521630 '2022-11-15 15:13:50' 1668521630 305 397] incT0: 1.0
agXYNn: [1668521631 '2022-11-15 15:13:51' 1668521631 292 401] incTN: 0.0
posX: 292.0 posY: 401.0
posX: 0.4949748743718593 posY: 0.5500821018062397
count: 1011
1668521632.0
agXY0: [1668521631 '2022-11-15 15:13:51' 1668521631 286 401] incT0: 1.0
agXYNn: [1668521632 '2022-11-15 15:13:52' 1668521632 273 404] incTN: 0.0
posX: 273.0 posY: 404.

posX: 178.33333333333337 posY: 128.00000000000003
posX: 0.20938023450586274 posY: 0.10180623973727426
count: 1040
1668521668.0
agXY0: [1668521666 '2022-11-15 15:14:26' 1668521666 181 126] incT0: 2.0
agXYNn: [1668521669 '2022-11-15 15:14:29' 1668521669 173 132] incTN: 1.0
posX: 175.66666666666669 posY: 130.00000000000003
posX: 0.2026800670016751 posY: 0.10509031198686376
count: 1041
1668521669.0
agXY0: [1668521666 '2022-11-15 15:14:26' 1668521666 181 126] incT0: 3.0
agXYNn: [1668521669 '2022-11-15 15:14:29' 1668521669 173 132] incTN: 0.0
posX: 173.0 posY: 132.0
posX: 0.19597989949748743 posY: 0.10837438423645321
count: 1042
1668521670.0
agXY0: [1668521669 '2022-11-15 15:14:29' 1668521669 173 133] incT0: 1.0
agXYNn: [1668521670 '2022-11-15 15:14:30' 1668521670 167 143] incTN: 0.0
posX: 167.0 posY: 143.0
posX: 0.18090452261306533 posY: 0.12643678160919541
count: 1043
1668521671.0
agXY0: [1668521670 '2022-11-15 15:14:30' 1668521670 167 144] incT0: 1.0
agXYNn: [1668521671 '2022-11-15 15:14:

In [10]:
print(X)
print(X.shape)


[[[0.57177616 0.57177616 0.57177616 ... 0.6837794  0.73163017 0.64720195]
  [0.60924574 0.73746959 0.51849148 ... 0.69618816 0.73163017 0.67250608]
  [0.62368208 0.73746959 0.56545012 ... 0.6783455  0.68442822 0.67323601]
  ...
  [0.35985401 0.35985401 0.35985401 ... 0.65024331 0.65790754 0.64257908]
  [0.48345499 0.60705596 0.35985401 ... 0.69416058 0.74574209 0.64257908]
  [0.74635036 0.88564477 0.60705596 ... 0.75425791 0.76277372 0.74574209]]

 [[0.54513382 0.57177616 0.51849148 ... 0.69221411 0.73163017 0.67250608]
  [0.60713706 0.73746959 0.51849148 ... 0.67672344 0.68442822 0.67250608]
  [0.57761557 0.59927007 0.56545012 ... 0.68491484 0.70413625 0.67323601]
  ...
  [0.48345499 0.60705596 0.35985401 ... 0.69416058 0.74574209 0.64257908]
  [0.74635036 0.88564477 0.60705596 ... 0.75425791 0.76277372 0.74574209]
  [0.80644769 0.88564477 0.72725061 ... 0.76642336 0.77007299 0.76277372]]

 [[0.60924574 0.73746959 0.51849148 ... 0.69618816 0.73163017 0.67250608]
  [0.62368208 0.737469

In [11]:
print(Y)
print(Y.shape)


[[0.66834171 0.12972085]
 [0.66331658 0.1592775 ]
 [0.64824121 0.22331691]
 ...
 [0.42713568 0.53858785]
 [0.49748744 0.58128079]
 [0.51256281 0.7044335 ]]
(1053, 2)


In [12]:
from tensorflow import keras
from tensorflow.keras import layers
from keras.optimizers import Adam
from keras.layers import MaxPooling2D, Activation, Dense
from tensorflow.keras.layers import Input, Conv1D, Conv2D, ReLU, BatchNormalization, Add, AveragePooling2D, Flatten, Dropout, LSTM, TimeDistributed, MaxPooling1D, Attention
from tensorflow.keras.models import Model
from sklearn.model_selection import KFold

#Dividimos los datos en K fold para cross validation
kf = KFold(n_splits=10,shuffle=True)
kf.get_n_splits(X)

#Un fichero para escribir los resultados de prediccion y reales ("borramos lo anterior")
f = open(flat+"xy.tsv", "w") 
f.write("")
f.close()

#Para cada K del cross validator   
for train_index, test_index in kf.split(X):
    #Datos de train y test
    print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_val = X[train_index], X[test_index]
    y_train, y_val = Y[train_index], Y[test_index]


    # Un nuevo modelo
    inputs = Input(shape=(X.shape[1], X.shape[2]))

    # 3 capas de convolucion 
    x = Conv1D(filters=16, kernel_size=2, activation='relu')(inputs)
    x = Conv1D(filters=32, kernel_size=3, activation='relu')(x)
    x = Conv1D(filters=64, kernel_size=3, activation='relu')(x)

    # 2 capas de LSTM de secuencia
    x = Dropout(0.5)(x)
    x = LSTM(32, return_sequences=True)(x)
    x = Dropout(0.5)(x)
    lstm_out = LSTM(32, return_sequences=True)(x)

    # Capa de atención
    attention_out = Attention()([lstm_out, lstm_out])

    # Aplanamos los datos y pasamos a una capa densa
    x = Flatten()(attention_out)
    x = Dense(512)(x)
    x = Dense(256)(x)
    outputs = Dense(2, activation="sigmoid")(x)

    model = Model(inputs, outputs)

    model.compile(loss='mean_squared_error', optimizer='adam')
    model.summary()

    #iteramos 1000 veces
    history = model.fit(x_train , y_train,
                        batch_size=64,
                validation_data=(x_val,y_val),
                epochs=1000)
    # Predecimos
    yhat = model.predict(x_val)

    #Desnormalizamos
    x1=y_val[:,0]*(max_x-min_x)+min_x
    y1=y_val[:,1]*(max_y-min_y)+min_y
    x2=yhat[:,0]*(max_x-min_x)+min_x
    y2=yhat[:,1]*(max_y-min_y)+min_y

    #calculamos error absoluto y ponemos indice, posiciones reales y predichas 
    absx=0.0
    absy=0.0

    f = open(flat+"xy.tsv", "a")
    for i in range(0,len(x2)):
        f.write(str(test_index[i])+"\t"+str(x1[i])+"\t"+str(y1[i])+"\t"+str(x2[i])+"\t"+str(y2[i])+"\n")
        absx=absx+abs(x1[i]-x2[i])
        absy=absy+abs(y1[i]-y2[i])
    f.close()

    absx=absx/(float)(len(x2))
    absy=absy/(float)(len(x2))

    print("absx:"+str(absx))
    print("absy:"+str(absy))

2024-06-25 14:11:12.796742: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-25 14:11:12.825747: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-25 14:11:13.512027: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


TRAIN: [   0    1    2    3    4    5    6    7    8    9   10   11   12   14
   15   16   17   18   19   20   21   22   23   24   25   26   27   28
   29   30   31   32   33   34   35   36   37   38   39   40   41   43
   46   47   48   49   50   51   52   54   55   56   58   59   60   62
   63   64   65   66   67   68   69   70   71   72   73   74   75   76
   77   78   79   81   82   83   84   85   86   87   88   89   91   92
   93   94   95   96   97   98   99  100  101  103  104  105  106  107
  108  109  110  111  112  113  114  115  116  117  118  119  120  121
  122  123  124  125  126  128  129  130  131  132  133  134  135  136
  137  138  139  140  141  142  143  144  145  146  147  148  149  150
  152  153  154  155  156  157  158  159  160  161  162  163  164  165
  166  167  168  169  170  171  172  173  174  176  177  178  181  182
  183  184  185  186  187  188  189  190  191  192  193  194  195  197
  198  199  200  201  202  203  204  205  206  207  208  209  210  211

2024-06-25 14:11:14.967799: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-25 14:11:14.991767: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-25 14:11:14.991853: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-25 14:11:14.993324: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-25 14:11:14.993374: I tensorflow/compile

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 8, 18)]      0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 7, 16)        592         ['input_1[0][0]']                
                                                                                                  
 conv1d_1 (Conv1D)              (None, 5, 32)        1568        ['conv1d[0][0]']                 
                                                                                                  
 conv1d_2 (Conv1D)              (None, 3, 64)        6208        ['conv1d_1[0][0]']               
                                                                                              

2024-06-25 14:11:17.004946: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-25 14:11:17.006441: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-25 14:11:17.007711: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

...

15/15 [==============================] - 0s 20ms/step - loss: 0.0019 - val_loss: 0.0026
Epoch 628/1000
15/15 [==============================] - 0s 20ms/step - loss: 0.0017 - val_loss: 0.0027
Epoch 629/1000
15/15 [==============================] - 0s 20ms/step - loss: 0.0017 - val_loss: 0.0026
Epoch 630/1000
15/15 [==============================] - 0s 20ms/step - loss: 0.0019 - val_loss: 0.0026
Epoch 631/1000
15/15 [==============================] - 0s 20ms/step - loss: 0.0019 - val_loss: 0.0028
Epoch 632/1000
15/15 [==============================] - 0s 20ms/step - loss: 0.0018 - val_loss: 0.0027
Epoch 633/1000
15/15 [==============================] - 0s 20ms/step - loss: 0.0017 - val_loss: 0.0028
Epoch 634/1000
15/15 [==============================] - 0s 20ms/step - loss: 0.0017 - val_loss: 0.0030
Epoch 635/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.0018 - val_loss: 0.0026
Epoch 636/1000
15/15 [==============================] - 0s 21ms/step - loss: 0.0018 - va

15/15 [==============================] - 0s 20ms/step - loss: 0.0020 - val_loss: 0.0029
Epoch 707/1000
15/15 [==============================] - 0s 20ms/step - loss: 0.0019 - val_loss: 0.0029
Epoch 708/1000
15/15 [==============================] - 0s 21ms/step - loss: 0.0017 - val_loss: 0.0024
Epoch 709/1000
15/15 [==============================] - 0s 21ms/step - loss: 0.0017 - val_loss: 0.0025
Epoch 710/1000
15/15 [==============================] - 0s 20ms/step - loss: 0.0018 - val_loss: 0.0026
Epoch 711/1000
15/15 [==============================] - 0s 21ms/step - loss: 0.0017 - val_loss: 0.0027
Epoch 712/1000
15/15 [==============================] - 0s 20ms/step - loss: 0.0017 - val_loss: 0.0025
Epoch 713/1000
15/15 [==============================] - 0s 20ms/step - loss: 0.0017 - val_loss: 0.0026
Epoch 714/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.0018 - val_loss: 0.0028
Epoch 715/1000
15/15 [==============================] - 0s 20ms/step - loss: 0.0017 - va

15/15 [==============================] - 0s 20ms/step - loss: 0.0016 - val_loss: 0.0028
Epoch 786/1000
15/15 [==============================] - 0s 20ms/step - loss: 0.0016 - val_loss: 0.0026
Epoch 787/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.0015 - val_loss: 0.0028
Epoch 788/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.0018 - val_loss: 0.0030
Epoch 789/1000
15/15 [==============================] - 0s 20ms/step - loss: 0.0019 - val_loss: 0.0028
Epoch 790/1000
15/15 [==============================] - 0s 20ms/step - loss: 0.0017 - val_loss: 0.0025
Epoch 791/1000
15/15 [==============================] - 0s 21ms/step - loss: 0.0017 - val_loss: 0.0032
Epoch 792/1000
15/15 [==============================] - 0s 20ms/step - loss: 0.0017 - val_loss: 0.0028
Epoch 793/1000
15/15 [==============================] - 0s 20ms/step - loss: 0.0017 - val_loss: 0.0026
Epoch 794/1000
15/15 [==============================] - 0s 20ms/step - loss: 0.0017 - va

15/15 [==============================] - 0s 20ms/step - loss: 0.0015 - val_loss: 0.0027
Epoch 865/1000
15/15 [==============================] - 0s 22ms/step - loss: 0.0016 - val_loss: 0.0026
Epoch 866/1000
15/15 [==============================] - 0s 20ms/step - loss: 0.0015 - val_loss: 0.0025
Epoch 867/1000
15/15 [==============================] - 0s 20ms/step - loss: 0.0016 - val_loss: 0.0025
Epoch 868/1000
15/15 [==============================] - 0s 20ms/step - loss: 0.0017 - val_loss: 0.0027
Epoch 869/1000
15/15 [==============================] - 0s 20ms/step - loss: 0.0015 - val_loss: 0.0026
Epoch 870/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.0015 - val_loss: 0.0028
Epoch 871/1000
15/15 [==============================] - 0s 20ms/step - loss: 0.0017 - val_loss: 0.0029
Epoch 872/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.0016 - val_loss: 0.0028
Epoch 873/1000
15/15 [==============================] - 0s 21ms/step - loss: 0.0016 - va

15/15 [==============================] - 0s 19ms/step - loss: 0.0014 - val_loss: 0.0029
Epoch 944/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.0015 - val_loss: 0.0028
Epoch 945/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.0015 - val_loss: 0.0029
Epoch 946/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.0014 - val_loss: 0.0026
Epoch 947/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.0016 - val_loss: 0.0029
Epoch 948/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.0013 - val_loss: 0.0029
Epoch 949/1000
15/15 [==============================] - 0s 20ms/step - loss: 0.0015 - val_loss: 0.0027
Epoch 950/1000
15/15 [==============================] - 0s 20ms/step - loss: 0.0013 - val_loss: 0.0026
Epoch 951/1000
15/15 [==============================] - 0s 20ms/step - loss: 0.0017 - val_loss: 0.0030
Epoch 952/1000
15/15 [==============================] - 0s 19ms/step - loss: 0.0018 - va